<a href="https://colab.research.google.com/github/camdsDS/Cesar_Portfolio/blob/main/Titanic_from_kaggle_Logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Desafio Kaggle: https://www.kaggle.com/c/titanic
import pandas as pd #para tratar os dados
from sklearn.preprocessing import MinMaxScaler #para normalizar os dados
from sklearn.model_selection import train_test_split #para dividir os dados de teste em um grupo para desenvolver o modelo e outro para validar o modelo
from sklearn.linear_model import LogisticRegression #para realizar a regressão logística
from sklearn.metrics import accuracy_score #para avaliar a acurácia do modelo

In [2]:
#lendo os arquivos de treino e teste
Titanic_data_Train = pd.read_csv('train.csv', index_col = 'PassengerId')
Titanic_data_Test = pd.read_csv('test.csv', index_col = 'PassengerId')

In [3]:
Titanic_data_Train.head()
#Para mais informações sobre o tipo de dado apresentado em cada coluna, visite:
#https://www.kaggle.com/c/titanic/data

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
#definindo o alvo, as features, o treino e o teste
y = Titanic_data_Train.Survived
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X_train = Titanic_data_Train[features]
X_test = Titanic_data_Test[features]

In [5]:
X_train.dtypes
#Temos apenas 3 tipos de dados: int64, float64 e object. Vamos, a partir deles,
#criar dois df: dados numéricos e dados categóricos
X_train_numericos = X_train.select_dtypes(exclude=['object'])
X_test_numericos = X_test.select_dtypes(exclude=['object'])

X_train_categoricos = X_train.select_dtypes('object')
X_test_categoricos = X_test.select_dtypes('object')

In [6]:
#agora vamos tratar os dados

In [7]:
#função que dropa colunas com uma porcentagem de valores miss superior a um
#valor P_miss arbitrário
#P_miss é a menor % tolerada de miss
def DropaMiss(df, P_miss=0):
  df_miss2 = ((df.isna().sum()/len(df))*100).to_frame().rename(columns={0:'P_miss'}).reset_index().sort_values('P_miss', ascending = False)
  df_miss = ((df.isna().sum()/len(df))*100).to_frame().rename(columns={0:'P_miss'}).reset_index()
  df_miss = df_miss[df_miss['P_miss'] <= P_miss]
  df = df[df_miss['index'].to_list()]
  return df,df_miss2

In [8]:
#usando a função DropaMiss com o df X_train e gerando o relatorio
#'relatorio_miss' vamos dropar, se existirem, colunas com
#mais de 80% de valores miss
X_train_numericos, relatorio_miss_N = DropaMiss(X_train_numericos, 80)
X_train_categoricos, relatorio_miss_C = DropaMiss(X_train_categoricos, 80)

In [9]:
relatorio_miss_N
relatorio_miss_C
#Há apenas duas colunas (Age e Embarked) no df treino com valores miss e
#nenhuma apresenta valor maior do que 80% de miss (logo, a função nada dropou)
#Não parece que o valor miss, nesses casos, signifique algo. Por isso, no caso
#da coluna numérica iremos substituir os dados miss pelo valor médio

,index,P_miss
1,Embarked,0.224467
0,Sex,0.000000


In [10]:
#Tratando miss numerico do train
valores_train_numericos = {'Age': X_train_numericos['Age'].mean()}
X_train_numericos = X_train_numericos.fillna(value = valores_train_numericos)

In [11]:
#Tratando miss numerico do test
X_test_numericos.isna().sum()
valores_test_numericos = {'Age': X_train_numericos['Age'].mean(), 'Fare': X_train_numericos['Fare'].mean()}
X_test_numericos = X_test_numericos.fillna(value = valores_test_numericos) 

In [12]:
#não iremos tratar os valores miss categorico porque
#iremos usar pandas.get_dummies, que os transforma em 0 0 ... 0.

In [13]:
#procurando por valores inconsistentes
X_train_numericos.Pclass.describe()
X_train_numericos.Age.describe()
X_train_numericos.SibSp.describe()
X_train_numericos.Parch.describe()
X_train_numericos.Fare.describe() #valores 0 para Fare, mas vamos manter no nosso treino
X_train_categoricos.Sex.describe()
X_train_categoricos.Embarked.describe()

count     889
unique      3
top         S
freq      644
Name: Embarked, dtype: object

In [14]:
#transformando categóricas em números
X_train_categoricos = pd.get_dummies(X_train_categoricos)
X_test_categoricos = pd.get_dummies(X_test_categoricos)

In [15]:
#comparando os dados categóricos do train e do test
colunas_train_categoricos = X_train_categoricos.columns.to_list()
colunas_test_categoricos = X_test_categoricos.columns.to_list()

comp = set(X_train_categoricos) - set(X_test_categoricos)
comp
#as colunas estão iguais

set()

In [16]:
#juntando os df numericos e o categóricos
abt_train = pd.merge(X_train_numericos, X_train_categoricos, how ='left', left_index = True, right_index = True)
abt_test = pd.merge(X_test_numericos, X_test_categoricos, how ='left', left_index = True, right_index = True)

In [17]:
#normalização dos dados
scaler = MinMaxScaler()
scaler.fit(abt_train)

scaler2 = MinMaxScaler()
scaler2.fit(abt_test)

X_train = pd.DataFrame(scaler.transform(abt_train), columns=abt_train.columns)
X_train.index = abt_train.index

X_test = pd.DataFrame(scaler.transform(abt_test), columns=abt_test.columns)
X_test.index = abt_test.index

In [18]:
#dividindo o treino em desenvolvimento e validação
X_desenv, X_valid, y_desenv, y_valid = train_test_split(X_train, y, train_size=0.8, test_size=0.2)

In [19]:
#Desenvolvendo o modelo de Regressão Logística
model = LogisticRegression()
model.fit(X_desenv, y_desenv)
accuracy_score(y_valid, model.predict(X_valid))

0.7877094972067039

In [20]:
#usando o modelo no Test
preds_test = model.predict(X_test)

In [21]:
#salvando a predição em um arquivo csv
output = pd.DataFrame({'PassengerId': X_test.index, 'Survived': preds_test})
output.to_csv('Submission_Titanic.csv', index=False)